In [17]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import statsmodels.api as sm
import statsmodels.formula.api as smf
np.random.seed(1)

from sklearn.model_selection import train_test_split

## Lab

In [6]:
x = np.random.randn(10,10)
x.shape

(10, 10)

In [19]:
# In this data set missing values are encoded as '?' so we will need to
#  pass that to the `na_values` parameter.

df = pd.read_csv('http://www-bcf.usc.edu/~gareth/ISL/Auto.csv'
                 , na_values='?').dropna() # drop columns with NaN
print("Shape of dataframe: " + str(df.shape))
df.head()

Shape of dataframe: (392, 9)


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


### Pandas test/train split method

Do not need to define your response (y) and predictors (X).

In [71]:
train = df.sample(frac=0.5, replace=False, random_state=1)
test = df.loc[df.index.difference(train.index)]
print(train.shape)
print(test.shape)

(196, 9)
(196, 9)


In [16]:
df.take(np.random.permutation(len(df))[:2])

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
271,23.2,4,156.0,105.0,2745,16.7,78,1,plymouth sapporo
147,24.0,4,90.0,75.0,2108,15.5,74,2,fiat 128


### Sklearn test/train split method

You must define your response (y) and predictors (X).

In [35]:
X = df.iloc[:,1:] #all columns except 'mpg'
y = df['mpg']
print(X.shape)
print(y.shape)

(392, 8)
(392,)


In [40]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=1)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(196, 8)
(196,)
(196, 8)
(196,)
